In [46]:
import pandas as pd
import numpy as np

In [47]:
df = pd.read_csv('M1-train.csv')
df.head()

,Timestamp,Bearing,BlockRef,CallDistanceAlongRoute,DataFrameRef,DatedVehicleJourneyRef,DestinationName,DestinationRef,DirectionRef,DistanceFromCall,...,PublishedLineName,RecordedAtTime,StopPointName,StopPointRef,StopsFromCall,VehicleRef,VisitNumber,DayOfWeek,VehicleDistanceAlongRoute,Segment
0,2018-04-06 16:00:01-04:00,233.35362,MTA NYCT_OF_B8-Weekday_C_OF_29220_M1-122,5169.49,2018-04-06,MTA NYCT_OF_B8-Weekday-092900_M1_145,GRAND STREET via 5 AV/BROADWAY,MTA_803192,1,23.58,...,M1,2018-04-06T16:00:01.000-04:00,5 AV/EAST 90 ST,MTA_400117,0,MTA NYCT_3891,1,5,5145.91,188189
1,2018-04-06 16:00:04-04:00,233.75014,MTA NYCT_OF_B8-Weekday_C_OF_20820_M1-105,8868.91,2018-04-06,MTA NYCT_OF_B8-Weekday-089200_M1_140,GRAND STREET via 5 AV/BROADWAY,MTA_803192,1,77.74,...,M1,2018-04-06T16:00:04.000-04:00,5 AV/W 45 ST,MTA_405179,0,MTA NYCT_3901,1,5,8791.17,1514
2,2018-04-06 16:00:07-04:00,234.19359,MTA NYCT_OF_B8-Weekday_C_OF_54660_M1-146,6718.93,2018-04-06,MTA NYCT_OF_B8-Weekday-091400_M1_146,E VILLAGE 8 ST via 5 AV,MTA_403793,1,48.54,...,M1,2018-04-06T16:00:07.000-04:00,5 AV/E 72 ST,MTA_400124,0,MTA NYCT_3898,1,5,6670.39,2216
3,2018-04-06 16:00:08-04:00,234.03671,MTA NYCT_OF_B8-Weekday_C_OF_28980_M7-204,8227.92,2018-04-06,MTA NYCT_OF_B8-Weekday-089800_M7_207,E VILLAGE 8 ST via 5 AV,MTA_403793,1,30.32,...,M1,2018-04-06T16:00:08.000-04:00,5 AV/W 53 ST,MTA_400514,0,MTA NYCT_3889,1,5,8197.60,1615
4,2018-04-06 16:00:11-04:00,233.93703,MTA NYCT_OF_B8-Weekday_C_OF_18120_M1-102,9554.96,2018-04-06,MTA NYCT_OF_B8-Weekday-088600_M7_225,GRAND STREET via 5 AV/BROADWAY,MTA_803192,1,2.96,...,M1,2018-04-06T16:00:11.000-04:00,5 AV/W 36 ST,MTA_400323,0,MTA NYCT_3903,1,5,9552.00,14118


In [48]:
seg = pd.read_csv('data/traffic-features-mim.csv')
seg

,segment_id,ID,from,to,segment,meter,feet,intersections,blocks,busstops
0,1,188189,188,189,5th Avenue - Southbound - 96th St to 86th St,811.76,2663.22,11,10,4
1,2,18922,189,22,5th Avenue - Southbound - 86th St to 72nd St,1153.06,3782.96,15,14,5
2,3,2216,22,16,5th Avenue - Southbound - 72nd St to 57th St,1207.64,3962.03,16,15,6
3,4,1615,16,15,5th Avenue - Southbound - 57th St to 49th St,642.36,2107.45,9,8,1
4,5,1514,15,14,5th Avenue - Southbound - 49th St to 42nd St,564.20,1851.03,8,7,3
5,6,14118,14,118,5th Avenue - Southbound - 42nd St to 34th St,642.87,2109.13,9,8,3


In [49]:
dff = df.merge(seg, left_on='Segment', right_on='ID', how='inner')

In [50]:
dff.head()

,Timestamp,Bearing,BlockRef,CallDistanceAlongRoute,DataFrameRef,DatedVehicleJourneyRef,DestinationName,DestinationRef,DirectionRef,DistanceFromCall,...,segment_id,ID,from,to,segment,meter,feet,intersections,blocks,busstops
0,2018-04-06 16:00:01-04:00,233.35362,MTA NYCT_OF_B8-Weekday_C_OF_29220_M1-122,5169.49,2018-04-06,MTA NYCT_OF_B8-Weekday-092900_M1_145,GRAND STREET via 5 AV/BROADWAY,MTA_803192,1,23.58,...,1,188189,188,189,5th Avenue - Southbound - 96th St to 86th St,811.76,2663.22,11,10,4
1,2018-04-06 16:00:32-04:00,233.91893,MTA NYCT_OF_B8-Weekday_C_OF_29220_M1-122,5398.92,2018-04-06,MTA NYCT_OF_B8-Weekday-092900_M1_145,GRAND STREET via 5 AV/BROADWAY,MTA_803192,1,107.21,...,1,188189,188,189,5th Avenue - Southbound - 96th St to 86th St,811.76,2663.22,11,10,4
2,2018-04-06 16:01:04-04:00,233.91893,MTA NYCT_OF_B8-Weekday_C_OF_29220_M1-122,5398.92,2018-04-06,MTA NYCT_OF_B8-Weekday-092900_M1_145,GRAND STREET via 5 AV/BROADWAY,MTA_803192,1,105.06,...,1,188189,188,189,5th Avenue - Southbound - 96th St to 86th St,811.76,2663.22,11,10,4
3,2018-04-06 16:01:34-04:00,234.37209,MTA NYCT_OF_B8-Weekday_C_OF_29220_M1-122,5398.92,2018-04-06,MTA NYCT_OF_B8-Weekday-092900_M1_145,GRAND STREET via 5 AV/BROADWAY,MTA_803192,1,4.29,...,1,188189,188,189,5th Avenue - Southbound - 96th St to 86th St,811.76,2663.22,11,10,4
4,2018-04-06 16:02:06-04:00,233.95068,MTA NYCT_OF_B8-Weekday_C_OF_29220_M1-122,5621.35,2018-04-06,MTA NYCT_OF_B8-Weekday-092900_M1_145,GRAND STREET via 5 AV/BROADWAY,MTA_803192,1,203.00,...,1,188189,188,189,5th Avenue - Southbound - 96th St to 86th St,811.76,2663.22,11,10,4


In [51]:
dff.shape

(12370, 43)

In [52]:
dff = dff[['Timestamp','VehicleRef', 'VehicleDistanceAlongRoute', 'segment_id']]

In [53]:
dff.head()

,Timestamp,VehicleRef,VehicleDistanceAlongRoute,segment_id
0,2018-04-06 16:00:01-04:00,MTA NYCT_3891,5145.91,1
1,2018-04-06 16:00:32-04:00,MTA NYCT_3891,5291.71,1
2,2018-04-06 16:01:04-04:00,MTA NYCT_3891,5293.86,1
3,2018-04-06 16:01:34-04:00,MTA NYCT_3891,5394.63,1
4,2018-04-06 16:02:06-04:00,MTA NYCT_3891,5418.35,1


M1 Bus Stops from: https://bustime.mta.info/m/;jsessionid=BF8A7BE70A434A7BA0468C768AC9E5A7?q=M1#0

In [54]:
#check number of buses in service on route
dff.VehicleRef.unique()

array(['MTA NYCT_3891', 'MTA NYCT_3897', 'MTA NYCT_6781', 'MTA NYCT_3838',
       'MTA NYCT_6779', 'MTA NYCT_6761', 'MTA NYCT_6771', 'MTA NYCT_6758',
       'MTA NYCT_6777', 'MTA NYCT_6770', 'MTA NYCT_6768', 'MTA NYCT_3890',
       'MTA NYCT_3902', 'MTA NYCT_3831', 'MTA NYCT_6748', 'MTA NYCT_3837',
       'MTA NYCT_6762', 'MTA NYCT_6749', 'MTA NYCT_6780', 'MTA NYCT_6784',
       'MTA NYCT_3889', 'MTA NYCT_3900', 'MTA NYCT_6778', 'MTA NYCT_6759',
       'MTA NYCT_3903', 'MTA NYCT_3894', 'MTA NYCT_6772', 'MTA NYCT_3834',
       'MTA NYCT_6789', 'MTA NYCT_3882', 'MTA NYCT_3881', 'MTA NYCT_3904',
       'MTA NYCT_3887', 'MTA NYCT_6765', 'MTA NYCT_6750', 'MTA NYCT_6776',
       'MTA NYCT_3886', 'MTA NYCT_3833', 'MTA NYCT_6767', 'MTA NYCT_6783',
       'MTA NYCT_3893', 'MTA NYCT_3829', 'MTA NYCT_6782', 'MTA NYCT_6763',
       'MTA NYCT_3888', 'MTA NYCT_6769', 'MTA NYCT_3895', 'MTA NYCT_3896',
       'MTA NYCT_3883', 'MTA NYCT_6752', 'MTA NYCT_3830', 'MTA NYCT_3899',
       'MTA NYCT_6775', '

In [55]:
dff['Timestamp'] = pd.to_datetime(dff['Timestamp']).dt.tz_localize('UTC').dt.tz_convert('America/New_York')

In [56]:
dff['Time'] = dff['Timestamp'].dt.strftime("%H:%M:%S")

In [57]:
dff['Date'] = dff['Timestamp'].dt.date

### Find Link Travel Time between segments

In [58]:
dff.head()

,Timestamp,VehicleRef,VehicleDistanceAlongRoute,segment_id,Time,Date
0,2018-04-06 16:00:01-04:00,MTA NYCT_3891,5145.91,1,16:00:01,2018-04-06
1,2018-04-06 16:00:32-04:00,MTA NYCT_3891,5291.71,1,16:00:32,2018-04-06
2,2018-04-06 16:01:04-04:00,MTA NYCT_3891,5293.86,1,16:01:04,2018-04-06
3,2018-04-06 16:01:34-04:00,MTA NYCT_3891,5394.63,1,16:01:34,2018-04-06
4,2018-04-06 16:02:06-04:00,MTA NYCT_3891,5418.35,1,16:02:06,2018-04-06


In [72]:
#records of buses from point closest/furthest to start of segment
dfo = dff.loc[dff.groupby(['VehicleRef', 'segment_id'])['VehicleDistanceAlongRoute'].idxmin()]
dfd = dff.loc[dff.groupby(['VehicleRef', 'segment_id'])['VehicleDistanceAlongRoute'].idxmax()]

In [73]:
#get difference between DistanceFromCall (y-x)
df_avg = dfo.merge(dfd, on=['VehicleRef', 'segment_id'])

In [74]:
df_avg.columns

Index([u'Timestamp_x', u'VehicleRef', u'VehicleDistanceAlongRoute_x',
       u'segment_id', u'Time_x', u'Date_x', u'Timestamp_y',
       u'VehicleDistanceAlongRoute_y', u'Time_y', u'Date_y'],
      dtype='object')

In [76]:
df_avg = df_avg[['Timestamp_x', 'VehicleRef', 'VehicleDistanceAlongRoute_x', 'VehicleDistanceAlongRoute_y', 'Timestamp_y', 'segment_id']]

In [77]:
df_avg['DistanceDelta'] = df_avg['VehicleDistanceAlongRoute_y'] - df_avg['VehicleDistanceAlongRoute_x']
df_avg['TimeDelta'] = df_avg['Timestamp_y'] - df_avg['Timestamp_x']
df_avg['TimeDelta'] = df_avg['TimeDelta'].astype('timedelta64[s]').astype(int)
df_avg['AvgSpeed'] = df_avg['DistanceDelta'] / df_avg['TimeDelta']

/Users/rachellim/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/rachellim/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/rachellim/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [85]:
df_avg.shape

(384, 9)

In [86]:
np.sum((df_avg['TimeDelta'] < 0).values.ravel())

101